# MIBACT Luoghi della cultura
## Data exploration


## Luoghi della cultura

I luoghi della cultura italiani raccolgono informazioni anagrafiche su: musei, aree archeologiche, monumenti, ville, etc. La banca dati viene manutenuta continuamente dal personale MiBAC e da personale e sistemi delle amministrazioni regionali.

Gli unique sul campo sono circa 6.604 a fronte di 52.392 record scaricati.

In [1]:
import pandas as pd
import numpy as np

### Script per estrazione luoghi cultura 

Per completezza lo si riporta all'interno del notebook.

inserire link a sorgente in repo

In [ ]:
import urllib.request
import time
import os
import json
import sys
import pysftp

print('Querying MIBACT endpoint for "Luoghi della cultura" dataset')
outfilepath = './data/mibact_luoghi_cultura_' + str(int(time.time())) + '.csv'

sparql_endpoint = 'http://dati.beniculturali.it/sparql?default-graph-uri=&'

# parametri per la paginazione
limit = 10000
offset = 0

# select per le query
select = 'select+*+'
select_count = 'select+count(*)+as+%3Fc+'

# corpo della query
where = 'where+{+select+distinct+%3Fs+as+%3Fsubject+%3FNome_Istituzionale+%3FDescrizione+%3FDescription+%3FISILIdentifier+%3FLatitudine+%3FLongitudine+%3FDisciplina+%3FTipo_luogo+%3FIndirizzo+%3FCodice_postale+%3FComune+%3FProvincia+%3FPrenotazioni+%3FOrari_di_apertura+%3FTelefono+%3FFax+%3FEmail+%3FWebSite+str%28%3FBiglietti%29+as+%3FBiglietti+%3FServizi+where+%7B+graph+%3Chttp%3A%2F%2Fdati.beniculturali.it%2Fmibact%2Fluoghi%3E+%7B+%3Fs+rdf%3Atype+cis%3ACulturalInstituteOrSite+%3B+cis%3AinstitutionalName+%3FNome_Istituzionale+.+optional+%7B+%3Fs+cis%3Adescription+%3FDescrizione+.+filter%28lang%28%3FDescrizione%29+%3D+%22it%22+%7C%7C+lang%28%3FDescrizione%29+%3D+%22%22%29+%7D+optional+%7B+%3Fs+cis%3Adescription+%3FDescription+.+filter%28lang%28%3FDescription%29+%3D+%22en%22%29+%7D+optional+%7B+%3Fs+cis%3AISILIdentifier+%3FISILIdentifier+%7D+optional+%7B+%3Fs+geo%3Alat+%3FLatitudine+%7D+optional+%7B+%3Fs+geo%3Along+%3FLongitudine+%7D+optional+%7B+%3Fs+dc%3Atype+%3FTipo_luogo+%7D+optional+%7B+%3Fs+cis%3AhasDiscipline+%5Bcis%3Aname+%3FDisciplina%5D+%7D+optional+%7B+%3Fs+cis%3AhasSite+%5Bcis%3AhasAddress+%3Faddress+%5D+.+optional+%7B+%3Faddress+cis%3AfullAddress+%3FIndirizzo+%7D+optional+%7B+%3Faddress+cis%3ApostCode+%3FCodice_postale+%7D+optional+%7B+%3Faddress+cis%3ApostName+%3FComune+%7D+optional+%7B+%3Faddress+cis%3AadminUnitL2+%3FProvincia+%7D+%7D+optional+%7B%3Fs+cis%3AhasAccessCondition+%5Brdf%3Atype+cis%3ABooking+%3B+cis%3Aname+%3FPrenotazioni%5D+%7D+optional+%7B%3Fs+cis%3AhasAccessCondition+%5Brdf%3Atype+cis%3AOpeningHoursSpecification+%3B+cis%3Adescription+%3FOrari_di_apertura+%5D+%7D+optional+%7B+%3Fs+cis%3AhasContactPoint+%3FcontactPoint+.+optional+%7B+%3FcontactPoint+cis%3AhasTelephone+%3FTelefono+%7D+optional+%7B+%3FcontactPoint+cis%3AhasFax+%3FFax+%7D+optional+%7B+%3FcontactPoint+cis%3AhasEmail+%3FEmail+%7D+optional+%7B+%3FcontactPoint+cis%3AhasWebSite+%3FWebSite+%7D+%7D+optional+%7B+%3Fs+cis%3AhasTicket+%3Fticket+.+%3Foffer+cis%3Aincludes+%3Fticket+%3B+cis%3AhasPriceSpecification+%5Bcis%3AhasCurrencyValue+%3FBiglietti%5D+%7D+optional+%7B+%3Fs+cis%3AprovidesService+%5Bcis%3Aname+%3FServizi%5D+%7D+%7D+%7D+order+by+%3Fs+}'

# formati della risposta
format_json ='&format=application%2Fjson&timeout=0'
format_csv ='&format=text%2Fcsv&timeout=0'

# interrogazione per recuperare il numero di elementi
query = 'query=' + select_count + where + format_json
with urllib.request.urlopen(sparql_endpoint + query) as response:
    res = response.read()
res_json = json.loads(res.decode("utf-8"))
record_count = res_json.get('results').get('bindings')[0].get('c').get('value')
print('Total number of records: ' + record_count)

# scarico le singole pagine
tempfilenames = []
while offset < int(record_count):
    query = 'query=' + select + where + 'limit+' + str(limit) + '+offset+' + str(offset) + format_csv
    filename = outfilepath + '.' + str(offset)
    tempfilenames.append(filename)
    print('Fetching {} record with offset {} in {}'.format(str(limit), str(offset), filename))
    #TODO: gestire eccezioni
    local_filename, headers = urllib.request.urlretrieve(sparql_endpoint + query, filename)
    offset += limit

# recupero la riga di intestazione dal primo file per poterla aggiungere al file finale di output
header = ''
with open(tempfilenames[0], 'r') as f:
    header = f.readline()

# unisco i file in un unico csv, saltando la riga di intestazione
with open(outfilepath, 'a') as outfile:
    outfile.write(header)
    for filename in tempfilenames:
        with open(filename, 'r') as infile:
            next(infile)
            for line in infile:
                outfile.write(line)
print('Results collected in ' + outfilepath)


### Carico il dataset estratto da script 

In [2]:
lc = pd.read_csv("../data/1_raw/mibact_luoghi_cultura_1533651167.csv")

/Users/mariaclaudiabodino/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
lc.columns

Index(['subject', 'Nome_Istituzionale', 'Descrizione', 'Description',
       'ISILIdentifier', 'Latitudine', 'Longitudine', 'Disciplina',
       'Tipo_luogo', 'Indirizzo', 'Codice_postale', 'Comune', 'Provincia',
       'Prenotazioni', 'Orari_di_apertura', 'Telefono', 'Fax', 'Email',
       'WebSite', 'Biglietti', 'Servizi'],
      dtype='object')

In [4]:
lc.ISILIdentifier.unique()

array([nan])

In [5]:
lc.shape

(52392, 21)

### Creo lista per Google Places basandoci  su "Nome_Istituzionale" e "Comune"

In [6]:
lc["Nome_Istituzionale_Comune"]= lc['Nome_Istituzionale'] + ","+ lc["Comune"]

In [7]:
#Pulitura campi
lc['Nome_Istituzionale_Comune'] = lc['Nome_Istituzionale_Comune'].str.replace("-",' ')
lc['Nome_Istituzionale_Comune'] = lc['Nome_Istituzionale_Comune'].str.replace('"',' ')
lc['Nome_Istituzionale_Comune']= lc['Nome_Istituzionale_Comune'].str.replace("\r\n",' ')
lc['Nome_Istituzionale_Comune'] = lc['Nome_Istituzionale_Comune'].str.replace("  "," ")
#TRIM spazi
lc['Nome_Istituzionale_Comune'] = lc['Nome_Istituzionale_Comune'].str.strip()

In [8]:
lc_ = lc['Nome_Istituzionale_Comune'].unique()

In [9]:
len(lc_)

6620

### Check su campo originale e campo derivato 

In [10]:
lc["Nome_Istituzionale"].value_counts()

Museo delle Civiltà                                                                  5632
Parco archeologico del Colosseo - Foro romano e Palatino                              800
Biblioteca Nazionale Centrale di Firenze                                              640
Castello Bufalini                                                                     576
Museo nazionale romano - Palazzo Massimo                                              576
Museo della Comunità ebraica di Trieste Carlo e Vera Wagner                           320
Museo delle Civiltà - Museo nazionale d’arte orientale 'Giuseppe Tucci'               320
Museo nazionale del Bargello - Museo delle Cappelle Medicee                           320
MUNDA - Museo nazionale d'Abruzzo                                                     288
Galleria nazionale dell'Umbria                                                        256
Pinacoteca di Brera                                                                   240
Mini Mu   

In [12]:
lc.Nome_Istituzionale_Comune.value_counts()

Museo delle Civiltà,Roma                                                                          5632
Parco archeologico del Colosseo  Foro romano e Palatino,Roma                                       800
Biblioteca Nazionale Centrale di Firenze,Firenze                                                   640
Museo nazionale romano  Palazzo Massimo ,Roma                                                      576
Castello Bufalini,San Giustino                                                                     576
Museo della Comunità ebraica di Trieste Carlo e Vera Wagner,Trieste                                320
Museo delle Civiltà  Museo nazionale d’arte orientale 'Giuseppe Tucci',Roma                        320
Museo nazionale del Bargello  Museo delle Cappelle Medicee,Firenze                                 320
MUNDA  Museo nazionale d'Abruzzo,L'Aquila                                                          288
Galleria nazionale dell'Umbria,Perugia                                   

In [13]:
lc[lc.Nome_Istituzionale=="Castello Bufalini"].head(2)

,subject,Nome_Istituzionale,Descrizione,Description,ISILIdentifier,Latitudine,Longitudine,Disciplina,Tipo_luogo,Indirizzo,...,Provincia,Prenotazioni,Orari_di_apertura,Telefono,Fax,Email,WebSite,Biglietti,Servizi,Nome_Istituzionale_Comune
48689,http://dati.beniculturali.it/mibact/luoghi/res...,Castello Bufalini,<p>Castello Bufalini nasce come fortilizio mil...,"<span id=""result_box"" lang=""en""><span title=""C...",NaN,43.54936,12.176403,Altro,Villa o palazzo di interesse storico o artistico,"via Largo Crociani, 3 - San Giustino",...,Perugia,Nessuna,Ottobre-Marzo: Lunedì-Sabato; Aprile-Settembre...,+39 075 856115,NaN,mailto:pm-umb@beniculturali.it,http://polomusealeumbria.beniculturali.it/?pag...,Castello 2 €; Parco 1 €,Archivio,"Castello Bufalini,San Giustino"
48690,http://dati.beniculturali.it/mibact/luoghi/res...,Castello Bufalini,<p>Castello Bufalini nasce come fortilizio mil...,"<span id=""result_box"" lang=""en""><span title=""C...",NaN,43.54936,12.176403,Altro,Villa o palazzo di interesse storico o artistico,"via Largo Crociani, 3 - San Giustino",...,Perugia,Nessuna,October-March: Monday-Saturday; April-Septembe...,+39 075 856115,NaN,mailto:pm-umb@beniculturali.it,http://polomusealeumbria.beniculturali.it/?pag...,Castello 2 €; Parco 1 €,Archivio,"Castello Bufalini,San Giustino"


In [14]:
lc_.shape

(6620,)

### Preparo lista con i vari luoghi in distinct della cultura per i quali dobbiamo abbinare un identificativo (Google places)

A fronte di un dataset iniziale con più di 50.000 record, sono 6620 i record che si distinguono per nome 

In [73]:
df = pd.DataFrame(lc_, columns = ["place"])

In [74]:
df.to_csv("../data/2_raw_geo/LuoghiCulturaUnique.csv", index=False)

### Suddivido la lista con i luoghi in più chunk 
per comodità di esecuzione per lo script che va a chiamare le API di Google Places, oltre che a salvare il csv con tutti gli elementi, lo suddivido in chunk 

In [104]:
df.shape

(6620, 1)

In [90]:
def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)

def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    return np.split(dfm, indices)

Shape: (1000, 1); RangeIndex(start=0, stop=1000, step=1)
Shape: (1000, 1); RangeIndex(start=1000, stop=2000, step=1)
Shape: (1000, 1); RangeIndex(start=2000, stop=3000, step=1)
Shape: (1000, 1); RangeIndex(start=3000, stop=4000, step=1)
Shape: (1000, 1); RangeIndex(start=4000, stop=5000, step=1)
Shape: (1000, 1); RangeIndex(start=5000, stop=6000, step=1)
Shape: (620, 1); RangeIndex(start=6000, stop=6620, step=1)


In [ ]:
chunks = split(df, 1000)

for c in chunks:
    c.to_csv("../data/2_raw_geo/LuoghiCulturaUnique"+str(c.index)+".csv", index=False)
    print("Shape: {}; {}".format(c.shape, c.index))

### API Google places

Eseguo lo steso script GooglePlaces.py in /src.

L'output è stato salvato nella cartella data/2_geocoded 

### Concateno i vai file generati dallo script in un unico dataframe che metterò in join con il dataset iniziale

In [15]:
import glob
luoghi_geo = pd.concat([pd.read_csv(f) for f in glob.glob('../data/2_raw_geocoded/*.csv')])

In [16]:
luoghi_geo.shape

(6620, 7)

In [106]:
luoghi_geo.head(4)

,Unnamed: 0,formatted_address,google_place_id,input_string,lat,lon,name
0,0,"Piazza Municipio, 1, 83021 Avella AV, Italia",ChIJfdZh2f5KOhMRpfBBkRLlfaY,"Antiquarium di Avella,Avella",40.960636,14.599655,Comune di Avella
1,1,"Località Poggio Renzo, 53043 Chiusi SI, Italia",ChIJZfVC_cZRKRMRkuJtL_tv3vw,"Necropoli di Poggio Renzo,Chiusi",43.030433,11.952198,Tomba della Scimmia
2,2,"Via Pieve di S. Siro, 4, 25044 Capo di ponte B...",ChIJTb93AmrKg0cRr3l2RKXceE0,Parco archeologico nazionale dei Massi di Cemm...,46.031448,10.338518,Parco archeologico nazionale dei Massi di Cemmo
3,3,"84047 Capaccio SA, Italia",ChIJN1Hb4xPjOxMROZaFrnw2xqg,Parco Archeologico di Paestum Area archeologi...,40.488640,14.969323,Heraion alla Foce del Sele


### Join tra dataset luoghi della cultura e dataset geonormalizzato 

I due campi su cui andare in join sono "Nome_Istituzionale_Comune" e "input_string"

In [17]:
lc.columns

Index(['subject', 'Nome_Istituzionale', 'Descrizione', 'Description',
       'ISILIdentifier', 'Latitudine', 'Longitudine', 'Disciplina',
       'Tipo_luogo', 'Indirizzo', 'Codice_postale', 'Comune', 'Provincia',
       'Prenotazioni', 'Orari_di_apertura', 'Telefono', 'Fax', 'Email',
       'WebSite', 'Biglietti', 'Servizi', 'Nome_Istituzionale_Comune'],
      dtype='object')

In [19]:
luoghi_geo.columns

Index(['Unnamed: 0', 'formatted_address', 'google_place_id', 'input_string',
       'lat', 'lon', 'name'],
      dtype='object')

In [20]:
luoghi = lc.merge(luoghi_geo, left_on="Nome_Istituzionale_Comune", right_on="input_string", how="inner")

### Check inner join 

In [27]:
luoghi_geo.shape

(6620, 7)

In [29]:
lc.shape

(52392, 22)

In [26]:
# dataset finale
luoghi.shape

(52392, 27)

In [23]:
luoghi.head(4).T

,0,1,2,3
subject,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...
Nome_Istituzionale,"Museo civico aufidenate ""Antonio De Nino""",Museo civico etnografico di Castel del Monte,Museo civico etnografico di Castel del Monte,Museo civico aufidenate
Descrizione,"<div>Il museo, fondato nel 1897, conserva i re...","L'antico borgo, sorto sulle pendici delle mont...","L'antico borgo, sorto sulle pendici delle mont...",<div>Il Museo Civico Aufidenate è ospitato nel...
Description,"<div>The museum, founded in 1897, contains the...","<div>The center of old town, built at the foot...","<div>The center of old town, built at the foot...",<div>Aufidena's Municipal Museum is set inside...
ISILIdentifier,NaN,NaN,NaN,NaN
Latitudine,41.7342,42.3648,42.3648,41.7843
Longitudine,14.0325,13.7267,13.7267,14.1083
Disciplina,Archeologia,Etnografia e antropologia,Etnografia e antropologia,Archeologia
Tipo_luogo,"Museo, galleria non a scopo di lucro e/o raccolta","Museo, galleria non a scopo di lucro e/o raccolta","Museo, galleria non a scopo di lucro e/o raccolta","Museo, galleria non a scopo di lucro e/o raccolta"
Indirizzo,"Viale Mansueto De Amicis, SNC - Alfedena","Via del Municipio, 5 - Castel del Monte","Via del Municipio, 5 - Castel del Monte","Contrada della Maddalena, SNC - Castel di Sangro"


### Elimino le colonne duplicate e non necessarie 
- Unnamed: 0	
- input_string

In [24]:
luoghi.drop(axis=1, columns=["Unnamed: 0", "input_string"], inplace=True)

In [25]:
luoghi.columns

Index(['subject', 'Nome_Istituzionale', 'Descrizione', 'Description',
       'ISILIdentifier', 'Latitudine', 'Longitudine', 'Disciplina',
       'Tipo_luogo', 'Indirizzo', 'Codice_postale', 'Comune', 'Provincia',
       'Prenotazioni', 'Orari_di_apertura', 'Telefono', 'Fax', 'Email',
       'WebSite', 'Biglietti', 'Servizi', 'Nome_Istituzionale_Comune',
       'formatted_address', 'google_place_id', 'lat', 'lon', 'name'],
      dtype='object')

### Check valori nulli Luoghi della cultura

In [1]:
luoghi.isnull.sum()

NameError: name 'luoghi' is not defined

### Salvo dataframe luoghi della cultura arricchito

In [ ]:
luoghi.to_csv("../data/3_processed/LuoghiCulturaGeo.csv", index=False)